In [8]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping
from pathlib import Path

In [9]:
# File paths
input_tiff = Path('../data/MULTISPECTRAL/germination/main/germination_main.tif')   # Path to the input GeoTIFF file
shapefile_mask = Path('../data/Shape/ge.shp') # Path to the shapefile mask
output_tiff = Path('../data/MULTISPECTRAL/germination/main/germination_main_cropped.tif') # Path to save the cropped GeoTIFF

In [13]:
# Step 1: Load the shapefile using GeoPandas
gdf = gpd.read_file(shapefile_mask)

# Step 2: Open the GeoTIFF file with Rasterio
with rasterio.open(input_tiff) as src:
    # Check if the GeoTIFF has a valid CRS
    if src.crs is None:
        # Manually assign the CRS if it's missing (you need to know the correct CRS)
        manual_crs = 'EPSG:4326 - WGS 84'  # Example CRS, replace with the actual CRS
        print(f"No CRS found. Assigning CRS: {manual_crs}")
        src_crs = manual_crs
    else:
        src_crs = src.crs

    # If the shapefile's CRS doesn't match the GeoTIFF's CRS, reproject it
    if gdf.crs != src_crs:
        print(f"Reprojecting shapefile from {gdf.crs} to {src_crs}")
        gdf = gdf.to_crs(src_crs)

    # Step 3: Extract the geometry of the shapefile as GeoJSON format
    shapefile_geom = [mapping(geom) for geom in gdf.geometry]

    # Step 4: Crop the GeoTIFF using the shapefile geometry
    cropped_image, cropped_transform = mask(src, shapefile_geom, crop=True)
    
    # Update metadata to reflect the new dimensions of the cropped image
    cropped_meta = src.meta.copy()
    cropped_meta.update({
        "driver": "GTiff",
        "height": cropped_image.shape[1],
        "width": cropped_image.shape[2],
        "transform": cropped_transform,
        "crs": src_crs  # Ensure the CRS is included in the output
    })

    # Step 5: Write the cropped image to a new file
    with rasterio.open(output_tiff, 'w', **cropped_meta) as dest:
        dest.write(cropped_image)

print(f"Cropped GeoTIFF saved to {output_tiff}")

No CRS found. Assigning CRS: EPSG:4326 - WGS 84
Reprojecting shapefile from EPSG:4326 to EPSG:4326 - WGS 84


c:\Users\user\anaconda3\envs\research_env\lib\site-packages\rasterio\__init__.py:368: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


CRSError: Invalid projection: EPSG:4326 - WGS 84: (Internal Proj Error: proj_create: crs not found: EPSG:4326 - WGS 84)